# 常见问题
`作者：Tina`
`时间：2018-05-08`

##  “cuda runtime error(2): out of memory”
由于我们经常在PyTorch中处理大量的数据，所以小错误会迅速导致你的程序耗尽你所有的GPU。幸运的是，这些案例中的修复程序通常很简单。

这里有一些常见的事情需要检查：
- 不要在你的训练循环中累积历史。默认情况下，涉及需要梯度的变量计算将保存历史。这意味着你应该避免在训练循环外的计算中使用这些变量。相反，你应该将变量分离或访问其底层数据。

  有时，可微变量的发生可能是不明显的。考虑以下训练循环（从[原始部分](https://discuss.pytorch.org/t/high-memory-usage-while-training/162)删减得到的）：

```python
total_loss = 0
for i in range(10000):
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output)
    loss.backward()
    optimizer.step()
    total_loss += loss
```

在这里，`total_loss`在训练循环中的历史是累积的，因为损失是一个具有autograd历史的可微变量。
可以通过`total_loss += float(loss)`来解决这个问题。

还可以参见其他的[例子](https://discuss.pytorch.org/t/resolved-gpu-out-of-memory-error-with-batch-size-1/3719)。

- 不要保留你不需要的张量和变量。如果你把一个张量或变量赋给一个局部变量，Python在超出范围后才会释放它。你可以通过使用`del x`释放。
 类似地，如果你将一个张量或变量赋给一个对象的成员变量，直到对象超出作用范围才会被释放。
 局部变量的范围可能比你想象的还大，例如：
 ```python
 for i in range(5):
    intermediate = f(input[i])
    result += g(intermediate)
 output = h(result)
 return output   
 ```
 上面`intermediate`即使在`h`执行的时候仍然存在，因为它的作用域出了循环的末端。为了更早地释放它，当完成关于它的计算时，你应该使用`del intermediate`。
 
- 不要在太大的序列上运行RNNs。在RNN中反向传播所需的内存量与RNN的长度线性相关。
- 不要使用太大的线性层。一个线性层`nn.Linear(m, n)`使用O(nm)的内存。也就是说，权重的内存需求与特征的数量是一样的。记住，你需要的内存至少是权重内存的两倍，因为你还需要储存梯度

## GPU内存没有被正确释放
PyTorch使用一个缓存内存分配器来加速内存分配。结果是，在`nvidia-smi`上所显示的值通常不会反映真实的内存使用情况，参见[内存管理](https://pytorch.org/docs/stable/notes/cuda.html#cuda-memory-management)。如果你的GPU内存没有被释放，即使在Python退出之后，很有可能是一些Python子进程仍然存在。你可以通过`ps -elf | grep python`找到这些子进程，手动结束`kill -9 [pid]`。

## 数据加载器返回相同的随机数
你可能使用了其他库来生成数据集中的随机数，例如，当工作子进程是通过fork启动时，NumPy的RNG被复制。参见[torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)学习如何正确地在data loader worker中使用它的`worker_init_fn`建立随机的种子。

## 循环神经网络无法使用数据并行
待补充，[详细内容](https://pytorch.org/docs/stable/notes/faq.html)